In [1]:
# -*- coding: utf-8 -*-
import os
import requests
import xlrd,xlwt
from xlutils.copy import copy

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# 设置常量
PATH = os.path.abspath(r"D:\玖隆\物贸公司\报表")
FILENAME = "钢厂价格对比表.xlsx"
SHEETNAME = "url_map"
OUTCOME_FILE = "钢厂价格对比表_每日抓取数据.xlsx"

BASE_FILE = os.path.join(PATH,FILENAME)
print(BASE_FILE)

NEED_NUM = 18  # 需要的记录数

CHROME_DRIVER = 'D:\Python\驱动\chromedriver_win32\chromedriver.exe'


D:\玖隆\物贸公司\报表\钢厂价格对比表.xlsx


In [3]:
nrows = 0 
ncols = 0 
data = None

In [4]:
def open_para_xlsx(path=PATH, filename=FILENAME, sheetname=SHEETNAME):
    # 打开Excel对应的参数需求表
    data = xlrd.open_workbook(os.path.join(path,filename))
    table = data.sheet_by_name(sheetname)  # 返回一个xlrd.sheet.Sheet()对象
    data.sheet_loaded(sheetname)  # 检查某个sheet是否导入完毕
    
    nrows = table.nrows  #获取该sheet中的有效行数
    ncols = table.ncols  # 有效列数
    return table

In [5]:
table = open_para_xlsx()

In [6]:
# 将参数数据加载进内存
para_dic = {}

for i in range(NEED_NUM):
    para_dic[int(table.cell_value(i+2,5))] = {"品名":table.cell_value(i+2,7),
                                         "二级入口网址":table.cell_value(i+2,1),
                                         "二级入口网址查找对应表名":table.cell_value(i+2,4),
                                         "每日数据明细条数":table.cell_value(i+2,6),
                                         "型号":table.cell_value(i+2,8),
                                         "规格":table.cell_value(i+2,9),
                                         "钢厂":table.cell_value(i+2,10),
                                         "CTR值":table.cell_value(i+2,11),
                                         "匹配原始数据值":None,
                                         "是否找到对应表名":False,
                                         "更新成功":False
                                        }

In [7]:
def get_specfic_table_link(table_content,match_title):
    match_a = table_content.findAll("a",title=match_title)  # 在获取到的页面的bs4对象上，继续找到对应a标签对象
    
    next_a_href_suffix = table_content.findAll("a",title=match_title)[0].get("href")
    if next_a_href_suffix.startswith("http"):
        next_a_href = next_a_href_suffix
    else:
        next_a_href = "https:" + next_a_href_suffix
    return next_a_href

In [8]:
def find_content_href(index,para_dic=para_dic):
    url = para_dic.get(index).get("二级入口网址")
    match_title = para_dic.get(index).get("二级入口网址查找对应表名")
    response = requests.get(url)
    html_doc = response.text 
    soup = BeautifulSoup(html_doc,"html.parser")
    table_content = soup.find("ul",class_="nlist")
    
    # 判断是否在第一页就能找到对应的 表名 a标签
    titles = table_content.get_text().replace("\t","").replace("\n","")
    
    if titles.find(match_title) == -1:
        # 找第二页
        find_index = soup.find("div",class_="page")
        page2 = find_index.find("a")
        if page2.contents[0] == "2":
            page2_change = find_index.find("a")["href"].split("-")[-1]
            page2_url = url.replace("1.html",page2_change)
            response2 = requests.get(page2_url)
            soup2 = BeautifulSoup(response2.text,"html.parser")
            table_content2 = soup2.find("ul",class_="nlist")
            titles2 = table_content2.get_text().replace("\t","").replace("\n","")
            if titles2.find(match_title) == -1:
                print("今日数据尚未更新")
                return para_dic
            else: 
                para_dic[index]["是否找到对应表名"] = True
                para_dic[index]["每日新增数据的网址"] = get_specfic_table_link(table_content2,match_title)
    else:
        # 如果找到了指定的表名
        # 则解析获取对应的a标签的href网址
        # 返回对应的表名的 href 字符串
        para_dic[index]["是否找到对应表名"] = True
        para_dic[index]["每日新增数据的网址"] = get_specfic_table_link(table_content,match_title) 
        return para_dic

In [9]:
# 获取对应的新增网址，所有网址信息都输出，则说明今天数据都更新了
for i in range(1,19):
    try:
        if i != 16:
            find_content_href(i)
    except Exception as e:
        print(e)

In [10]:
for k,v in para_dic.items():
    
    localtime = time.localtime(time.time())
    if v.get("每日新增数据的网址") is not None:
        extract_month = int(v.get("每日新增数据的网址").split("/")[5][:2])
        
        if extract_month == localtime.tm_mon:
            print(k,v.get("是否找到对应表名"),v.get("每日新增数据的网址"))
        else: 
            para_dic.get(k)["是否找到对应表名"] = False
            para_dic.get(k)["每日新增数据的网址"] = None
            print(k,v.get("是否找到对应表名"),v.get("每日新增数据的网址"),"不是当月网址")
    else:
        print(k,v.get("是否找到对应表名"),v.get("每日新增数据的网址"))
        

1 True https://jiancai.mysteel.com/m/20/0601/09/BECB369348D80BE0.html
2 True https://jiancai.mysteel.com/m/20/0601/12/879A04F465FB5C25.html
3 True https://jiancai.mysteel.com/m/20/0601/10/AA6E267349F88E80.html
4 True http://youhanxian.mysteel.com/m/20/0601/11/7525E5D00609C38D.html
5 True http://youhanxian.mysteel.com/m/20/0601/10/65D0C8B3F5FE29CD.html
6 True http://youhanxian.mysteel.com/m/20/0601/10/1083DCECC3AD6591.html
7 True http://lengdun.mysteel.com/m/20/0601/10/3ADA9968198EEBD0.html
8 True http://lengdun.mysteel.com/m/20/0601/10/E51850D879909B25.html
9 True http://lengdun.mysteel.com/m/20/0601/11/3D3D5FCFE56594E6.html
10 True https://rezha.mysteel.com/m/20/0601/09/1EB23D45E8238AD3.html
11 True https://rezha.mysteel.com/m/20/0601/10/530926BDC2B272AD.html
12 True https://rezha.mysteel.com/m/20/0601/10/7890156DCEC43B1B.html
13 True https://zhongban.mysteel.com/m/20/0601/10/D7A38FD73FBC3C9A.html
14 True https://zhongban.mysteel.com/m/20/0601/10/1B8E807D31A1DC5D.html
15 True https://

# 最关键的价格数据是ajax返回的，通过selenium模拟服务器，获取ajax数据

In [11]:
#指定chrome的驱动
#执行到这里的时候Selenium会到指定的路径将chrome driver程序运行起来
driver = webdriver.Chrome(CHROME_DRIVER)
if para_dic.get(1).get("每日新增数据的网址") is not None:
    driver.get(para_dic.get(1).get("每日新增数据的网址"))
else:
    driver.get("https://jiancai.mysteel.com/m/20/0527/09/9C38A68E4E18B9D3.html")
    
time.sleep(10)
for k,v in para_dic.items():
    #get 方法 打开指定网址
    if k != 16:
        if para_dic.get(k).get("每日新增数据的网址") is not None:
            driver.get(para_dic.get(k).get("每日新增数据的网址"))
            id_ctr_value = para_dic.get(k).get("CTR值")
            #选择网页元素
            element_keyword = driver.find_element_by_id(id_ctr_value)
            para_dic[k]["匹配原始数据值"]=element_keyword.text
    time.sleep(1)

In [12]:
for k,v in para_dic.items():
    print(k,v.get("匹配原始数据值"))

1 螺纹钢 Φ16-25 HRB400 沙钢 3670 +20 Ф16:3700;Ф25:3720 代理
2 螺纹钢 Ф16 HRB400 韶钢 4040 +40 现货
3 螺纹钢 Ф16 HRB400E 河钢承钢 3730 +40 现货
4 硬线 45# 热轧 Φ6.5 沙钢 3820 +50 现货
5 硬线 45#-70# 热轧 Φ6.5-20 中天钢铁 3860 +30 现货
6 硬线 45# 热轧 Φ6.5 潍坊特钢 3710 +30 现货
7 冷镦钢 ML08AL 热轧 Φ16-24 沙钢 3910 +60 代理
8 冷镦钢 ML08AL 热轧 Φ6.5 中天钢铁 3880 +30 货少 现货
9 冷镦钢 ML08AL 热轧 Φ6.5-20 鞍钢 3890 +30 现货
10 热轧板卷 5.75*1500*C Q235B 沙钢 3650 +60 货少 现货
11 热轧板卷 5.5-11.5*1500*C Q235B 承钢 3700 +60 现货
12 热轧板卷 5.75*1500*C Q235B 河钢承钢 3700 +50 现货
13 普中板 12 Q235B 沙钢 3950 +40 现货
14 普中板 14-20 Q235B 鞍钢 3910 - 代理
15 普中板 14-20 Q235B 萍钢 3800 +40 现货
16 None
17 冷轧卷 1.0*1250*C DC01 本钢 3990 +50 现货
18 冷轧卷 1.0*1250*C SPCC 马钢 4080 +30 现货


# 将结果写入到excel文件中
由于不知道mysteel网站数据会不会发生较大的变化，写的太全面，后期不容易发现和修改，索性只要负责取数据。   
数据保存和校验全部在excel中完成

In [13]:
# 打开工作表，保持原表的格式
rb = xlrd.open_workbook(os.path.join(PATH,OUTCOME_FILE))
wb = copy(rb)
ws = wb.get_sheet("sheet1")

for i in range(1,19):
    if i != 16:
        ws.write(i-1,0,para_dic.get(i).get("是否找到对应表名"))
        ws.write(i-1,1,para_dic.get(i).get("匹配原始数据值"))
    
wb.save(os.path.join(PATH,OUTCOME_FILE)) 

由于xlrd和xlwt实在是很鸡肋的用法，而对xlsx格式的表格的支持比较差。
xlutils也不能支持过多的excel格式，只能写到新表中，手动打开复制